In [9]:
# general imports
import matplotlib.pyplot as plt
%matplotlib inline

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, Instruction, circuit, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
import numpy as np

import functools
import time

from quil_utils import *
from pyquil.quil import Pragma, Program
from pyquil.api import get_qc,QVM
from pyquil.gates import *
from quil_utils import Compiled_Circuit

#useful decorator for measuring local execution time
#https://realpython.com/primer-on-python-decorators/

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [3]:
def general_bell(dim):

    out = Circuit().h([0])

    for i in range(dim-1):
        out.cnot(0,i+1)

    return out

def long_hadamard(dim,out):

    for i in range(dim):
        out.h([i])

    return out


In [4]:
#25 massimo sulla mia macchina -EA
#bell = general_bell(50)
#print(bell)
ha = Circuit()
for i in range(30):
    long_hadamard(26,ha)
print(ha)

device = LocalSimulator(backend = "braket_sv")

T   : |0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|
                                                                                       
q0  : -H-H-H-H-H-H-H-H-H-H-H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--
                                                                                       
q1  : -H-H-H-H-H-H-H-H-H-H-H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--
                                                                                       
q2  : -H-H-H-H-H-H-H-H-H-H-H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--
                                                                                       
q3  : -H-H-H-H-H-H-H-H-H-H-H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--
                                                                                       
q4  : -H-H-H-H-H-H-H-H-H-H-H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--H--
                                

In [5]:
#decorate run function with a timer
device.run = timer(device.run)

task = device.run(ha, shots = 10000)
counts = task.result().measurement_counts

print(counts)

Finished 'run' in 364.3222 secs
Counter({'00000000000000000000000000': 10000})


In [25]:
qc = get_qc(name='Aspen-M-2',as_qvm=True,compiler_timeout=60*1000)


In [7]:
prog = transpile_braket_to_quil(ha)

In [8]:
compiled = Compiled_Circuit (device=qc ,program=prog)

In [26]:
qc.run(prog)

ReadTimeout: timed out